In [1]:
import math
import random
import os.path as osp

import os
import pandas as pd

import numpy as np

import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parameter import Parameter

from torch.utils.data import Dataset as BaseDataset
from torch.utils.data import DataLoader

from tqdm import tqdm
from datetime import datetime

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, ndcg_score, recall_score
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [2]:
import utils

In [3]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, dilation=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels,
                               kernel_size=3, padding=1, stride=stride)
        self.conv2 = nn.Conv2d(out_channels, out_channels,
                               kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=out_channels)
        self.bn2 = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU(inplace=True)

        # one kernel make one feature map
        # kernel 厚度 = channel 數
        self.is_changed = in_channels != out_channels
        self.trans = nn.Conv2d(in_channels, out_channels,
                               kernel_size=1, stride=stride)

    def forward(self, x):
        f_x = self.conv1(x)
        f_x = self.bn1(f_x)
        f_x = self.relu(f_x)
        f_x = self.conv2(f_x)
        f_x = self.bn2(f_x)

        if self.is_changed:
            x = self.trans(x)

        x = f_x + x
        x = self.relu(x)
        return x


class BottleNeck(nn.Module):

    def __init__(self, in_channels, out_channels, stride=1, dilation=4):
        super(BottleNeck, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=stride, padding=0)
        self.conv2 = nn.Conv2d(
            in_channels=out_channels, out_channels=out_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(
            in_channels=out_channels, out_channels=dilation*out_channels, kernel_size=1, padding=0)

        self.bn1 = nn.BatchNorm2d(num_features=out_channels)
        self.bn2 = nn.BatchNorm2d(num_features=out_channels)
        self.bn3 = nn.BatchNorm2d(num_features=out_channels * dilation)

        self.relu = nn.ReLU(inplace=True)

        # one kernel make one feature map
        # kernel 厚度 = channel 數
        self.is_changed = in_channels != (out_channels*dilation)
        self.trans = nn.Conv2d(in_channels, out_channels *
                               dilation, kernel_size=1, stride=stride)

    def forward(self, x):

        f_x = self.conv1(x)
        f_x = self.bn1(f_x)
        f_x = self.relu(f_x)
        f_x = self.conv2(f_x)
        f_x = self.bn2(f_x)
        f_x = self.relu(f_x)
        f_x = self.conv3(f_x)
        f_x = self.bn3(f_x)

        if self.is_changed:
            x = self.trans(x)

        x = f_x + x
        x = self.relu(x)
        return x


class _ResNet(nn.Module):
    def __init__(self, block, block_cnts, dilation=1):
        super(_ResNet, self).__init__()

        self.in_channels = 64
        self.out_channels = 64

        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=self.in_channels, kernel_size=7, stride=2, padding=3)
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._layer(
            block, block_cnts[0], dilation, self.in_channels, self.out_channels, stride=1)
        self.layer2 = self._layer(
            block, block_cnts[1], dilation, self.in_channels, self.out_channels, stride=2)
        self.layer3 = self._layer(
            block, block_cnts[2], dilation, self.in_channels, self.out_channels, stride=2)
        self.layer4 = self._layer(
            block, block_cnts[3], dilation, self.in_channels, self.out_channels, stride=2)

        self.avg = nn.AdaptiveAvgPool2d(output_size=(1, 1))

        self.dense = nn.Linear(in_features=self.in_channels, out_features=6)
        self.activation = nn.Softmax()
        self.flatten = nn.Flatten(start_dim=1)

    def _layer(self, block, block_cnt, dilation, in_channels, out_channels, stride):
        # in_channels: param of previous block output channel
        # out_channels: param of current block input channel

        blocks = []
        blocks.append(
            block(in_channels=in_channels,
                  out_channels=out_channels, stride=stride)
        )

        for cnt in range(1, block_cnt):
            b = block(in_channels=dilation * out_channels,
                      out_channels=out_channels)
            blocks.append(b)

        self.in_channels = out_channels * dilation
        self.out_channels = out_channels * 2

        return nn.Sequential(*blocks)

    def forward(self, x):
        x = self.conv1(x)
        x = self.max_pool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avg(x)

        x = self.flatten(x)
        x = self.dense(x)

        x = self.activation(x)
        return x

In [4]:
device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:1'

root = os.path.join(os.getcwd(), "Yelp")
rel_p = os.path.join(root, "user_business.dat")

user_cnt = 16239
item_cnt = 14284

cuda ready...


In [5]:
class Args:
    def __init__(self,
                 user_cnt,
                 item_cnt,
                embed_dim=50,
                inp_drop=0.5,
                feat_drop=0.5,
                hid_drop=0.2,
                perm=1,
                k_w=5,
                k_h=10,
                ker_sz=3):
        self.user_cnt = user_cnt
        self.item_cnt = item_cnt
        self.embed_dim = embed_dim
        self.inp_drop = inp_drop
        self.feat_drop = feat_drop
        self.hid_drop = hid_drop
        self.perm = perm
        self.k_w = k_w
        self.k_h = k_h
        self.ker_sz = ker_sz

# Yelp

In [6]:
compliment, compliment_i, compliment_m = utils.m2m_to_list(os.path.join(os.getcwd(), "Yelp", "user_compliment.dat"), user_cnt=user_cnt)
user, user_i, user_m = utils.m2m_to_list(os.path.join(os.getcwd(), "Yelp", "user_user.dat"), user_cnt=user_cnt)
# user, user_i, user_m = utils.m2m_to_list(os.path.join(os.getcwd(), "Movielens", "user_user.dat"), user_cnt=user_cnt)

In [7]:
city, city_i, city_m = utils.m2m_to_list(os.path.join(os.getcwd(), "Yelp", "business_city.dat"), user_cnt=14284)
category, category_i, category_m = utils.m2m_to_list(os.path.join(os.getcwd(), "Yelp", "business_category.dat"), user_cnt=14284)
# author, author_i, author_m = utils.m2m_to_list(os.path.join(os.getcwd(), "Movielens", "book_author.dat"), user_cnt=user_cnt)

## Interact

In [8]:
sparse_features = ["cols_0", "cols_1"] # user_id, movie_id
rating = "cols_2"

In [9]:
rel = utils.read_file(rel_p)
rel.head()

,cols_0,cols_1,cols_2
0,1,8391,5
1,1,8971,5
2,2,186,5
3,2,205,5
4,2,209,4


In [10]:
rel_feat, rel_pos = utils.get_explicit_features(rel_p)

In [11]:
user_cnt, item_cnt = rel_feat.shape

In [12]:
class Dataset(BaseDataset):
    
    def __init__(self, dataframe):
        
        # Read node features
        self.users = dataframe.cols_0.values
        self.items = dataframe.cols_1.values
        self.rats = dataframe.cols_2.values
        
    def __len__(self):
        return len(self.rats)
        
    def __getitem__(self, idx):
        
        user = self.users[idx]
        item = self.items[idx]
        rat = self.rats[idx]
        label = np.zeros(6)
        label[rat] = 1
        
        return user, item, rat, label

In [13]:
args = Args(user_cnt, item_cnt)

In [14]:
def get_chequer_perm(perm=1, k_w=30, k_h=50):
    
    embed_dim = k_w * k_h
    ent_perm  = np.int32([np.random.permutation(embed_dim) for _ in range(perm)])
    rel_perm  = np.int32([np.random.permutation(embed_dim) for _ in range(perm)])

    comb_idx = []
    for k in range(perm):
        temp = []
        ent_idx, rel_idx = 0, 0

        for i in range(k_h):
            for j in range(k_w):
                if k % 2 == 0:
                    if i % 2 == 0:
                        temp.append(ent_perm[k, ent_idx]); ent_idx += 1;
                        temp.append(rel_perm[k, rel_idx]+embed_dim); rel_idx += 1;
                    else:
                        temp.append(rel_perm[k, rel_idx]+embed_dim); rel_idx += 1;
                        temp.append(ent_perm[k, ent_idx]); ent_idx += 1;
                else:
                    if i % 2 == 0:
                        temp.append(rel_perm[k, rel_idx]+embed_dim); rel_idx += 1;
                        temp.append(ent_perm[k, ent_idx]); ent_idx += 1;
                    else:
                        temp.append(ent_perm[k, ent_idx]); ent_idx += 1;
                        temp.append(rel_perm[k, rel_idx]+embed_dim); rel_idx += 1;

        comb_idx.append(temp)

    chequer_perm = torch.LongTensor(np.int32(comb_idx)).to(device)
    return chequer_perm


In [15]:
chequer = get_chequer_perm(perm=args.perm, k_w=args.k_w, k_h=args.k_h)
chequer.shape

torch.Size([1, 100])

In [16]:
chequer

tensor([[ 2, 56, 46, 79, 16, 70, 23, 61, 27, 73, 59, 17, 89, 45, 80,  5, 63, 47,
         88, 43, 40, 86,  8, 51, 31, 66, 20, 57, 28, 74, 62, 21, 67, 37, 58, 35,
         87,  3, 68,  1,  4, 91, 29, 60, 12, 52,  6, 71, 14, 69, 76, 15, 96, 34,
         55,  9, 97, 44, 98, 10, 36, 94,  7, 90, 13, 93, 25, 65, 48, 92, 53, 19,
         50, 38, 78, 42, 83, 18, 72, 33, 30, 75, 22, 82, 24, 64, 49, 54, 11, 99,
         95,  0, 85, 39, 81, 32, 84, 41, 77, 26]], device='cuda:1')

In [17]:
class InteractE(torch.nn.Module):
    """
    Proposed method in the paper. Refer Section 6 of the paper for mode details 
    Parameters
    ----------
    params:         Hyperparameters of the model
    chequer_perm:   Reshaping to be used by the model

    Returns
    -------
    The InteractE model instance

    """
    def __init__(self, params, chequer_perm):
        super(InteractE, self).__init__()

        self.p = params
        self.user_embed = torch.nn.Embedding(self.p.user_cnt+1, self.p.embed_dim); # xavier_normal_(self.ent_embed.weight)
        self.item_embed = torch.nn.Embedding(self.p.item_cnt+1, self.p.embed_dim);
        self.chequer_perm = chequer_perm


        self.resnet = _ResNet(BasicBlock, [2, 2, 2, 2], 1)
    
    def circular_padding_chw(self, batch, padding):
        upper_pad = batch[..., -padding:, :]
        lower_pad = batch[..., :padding, :]
        temp = torch.cat([upper_pad, batch, lower_pad], dim=2)

        left_pad = temp[..., -padding:]
        right_pad = temp[..., :padding]
        padded = torch.cat([left_pad, temp, right_pad], dim=3)
        return padded

    def forward(self, user_idx, item_idx):

        user = self.user_embed(user_idx)
        item = self.item_embed(item_idx)
        
        comb_emb = torch.cat([user, item], dim=-1)
        chequer_perm = comb_emb[:, self.chequer_perm] # batch, 1, embed_size
 
        stack_inp = chequer_perm.reshape((-1, self.p.perm, 2*self.p.k_w, self.p.k_h)) # batch, 1, 2*k_w, k_h
#         print(user_idx, item_idx, chequer_perm.shape, stack_inp.shape)
        #### start
        x = self.circular_padding_chw(stack_inp, self.p.ker_sz//2)
        
        pred = self.resnet(x)

        return pred


In [18]:
def train(model, optim, loader, device):
    
    model.train()
    model = model.to(device)
    
    total_loss = 0
    for index, data in tqdm(enumerate(loader)):
        
        optim.zero_grad()
        
        user, item, label, _ = data
        
        user = user.to(device)
        item = item.to(device)
        label = label.to(device, dtype=torch.long)

        pred = model(user, item) 
        
        loss = torch.nn.CrossEntropyLoss()(pred, label)

        loss.backward()
        optim.step()
        
        total_loss += loss.item()

    total_loss = total_loss/(index+1)
    return total_loss

In [41]:
@torch.no_grad()
def test(model, loader, device="cpu"):
    
    from sklearn.metrics import mean_squared_error, ndcg_score, recall_score
    
    model.eval()
    model = model.to(device)
    
    mse_list = []
    recall_list = []
    ndcg_list = []
    for data in tqdm(loader):
        
        user, item, label, onehot = data
        user = user.to(device)
        item = item.to(device)

        pred = model(user, item) 
        mse = mean_squared_error(onehot, pred.detach().cpu().numpy())
        mse_list.append(mse)
        
        pred_val = torch.argmax(pred, dim=1).cpu().detach().numpy()
        pred_val = np.where(pred_val > 3, 1, 0).reshape(1, -1)
        label = np.where(label > 3, 1, 0).reshape(1, -1)
        
#         print(label, pred_val)
        recall = recall_score(label.reshape(-1, 1), pred_val.reshape(-1, 1))
        recall_list.append(recall)
        ndcg = ndcg_score(label, pred_val)
        ndcg_list.append(ndcg)
     
    return sum(mse_list)/len(mse_list), sum(recall_list)/len(recall_list), sum(ndcg_list)/len(ndcg_list)

In [42]:
@torch.no_grad()
def predict(model, loader, device="cpu"):
    
#     from sklearn.metrics import roc_auc_score
    
    model.eval()
    model = model.to(device)
    
    y_pred = []
    for data in tqdm(loader):
        follower, followee, _, _ = data
        
        user, item, label = data
        user = user.to(device)
        item = item.to(device)
        label = label.to(device)

        pred = model(user, item)
        
        y_pred.append(pred.view(-1).cpu())
     
    return torch.cat(y_pred)

In [43]:
k = 5
kf = KFold(n_splits=5)

In [ ]:
fold_cnt = 0

mse_list = []
recall_list = []
ndcg_list = []

for train_index, test_index in kf.split(rel):
    
    fold_cnt += 1
    print("========= Fold: {} ==========".format(fold_cnt))
    
    test_df = rel.iloc[test_index]
    
    train_index, valid_index = train_test_split(train_index, test_size=0.1)
    train_df = rel.iloc[train_index]
    valid_df = rel.iloc[valid_index]
    
    trainset = Dataset(train_df)
    validset = Dataset(valid_df)
    testset = Dataset(test_df)
    
    trainloader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=2) 
    testloader = DataLoader(testset, batch_size=256, shuffle=False, num_workers=2) 
    validloader = DataLoader(validset, batch_size=256, shuffle=False, num_workers=2) 
    
    model = InteractE(args, chequer).to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    
    for epoch in range(10):
        print("======= Epoch {} ========".format(epoch+1))
        loss = train(model, optimizer, trainloader, device)
        mse, recall, ndcg = test(model, trainloader, device)
        print("Trainset: mse={:.5f}, recall={:.5f}, ndcg={:.5f}".format(mse, recall, ndcg))
        mse, recall, ndcg = test(model, validloader, device)
        print("Validset: mse={:.5f}, recall={:.5f}, ndcg={:.5f}".format(mse, recall, ndcg))
        
    
    mse, recall, ndcg = test(model, testloader, device)
    mse_list.append(mse)
    recall_list.append(recall)
    ndcg_list.append(ndcg)

========= Fold: 1 ==========
======= Epoch 1 ========


0it [00:00, ?it/s]/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
558it [00:22, 24.39it/s]
  0%|          | 1/558 [00:00<01:27,  6.35it/s]/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/e

/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  8%|▊         | 46/558 [00:00<00:05, 96.33it/s]/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipyke

 15%|█▍        | 82/558 [00:00<00:04, 107.98it/s]/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipyk

/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
 23%|██▎       | 129/558 [00:01<00:03, 108.14it/s]/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipy

...

/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
 62%|██████▏   | 345/558 [00:03<00:01, 116.28it/s]/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipy

/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
 68%|██████▊   | 381/558 [00:03<00:01, 115.74it/s]/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipy

/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
 75%|███████▍  | 417/558 [00:03<00:01, 115.69it/s]/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipy

 81%|████████  | 453/558 [00:04<00:00, 115.39it/s]/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipy

/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ai2019/ne6081064/anaconda3/envs/june/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dime

In [132]:
valid_auc = 0
for epoch in range(100):

    loss = train(model, optimizer, trainloader, device)
    train_auc = test(model, trainloader, device)
    auc = test(model, validloader, device)
   
    if auc > valid_auc:
        valid_auc = auc
        
        checkpoint = {
            'model_stat': model.state_dict(),
            'optimizer_stat': optimizer.state_dict(),
        }
        
        torch.save(checkpoint, os.path.join(root, "results", dataset, "interactE", 
                                                "{}_{}.pth".format(date_time, dataset)))
        print("\nSave Model")
    
    print("Epoch: {}, loss={:.5f}, valid_auc={:.5f}, train_auc={:.5f}".format(epoch+1, loss, auc, train_auc))
    

408it [00:09, 44.31it/s]
100%|██████████| 136/136 [00:00<00:00, 137.70it/s]



Save Model
Epoch: 1, loss=0.73768, valid_auc=0.50883, train_auc=0.54036


408it [00:09, 44.89it/s]
100%|██████████| 136/136 [00:00<00:00, 140.28it/s]



Save Model
Epoch: 2, loss=0.70776, valid_auc=0.53263, train_auc=0.60736


408it [00:09, 42.33it/s]
100%|██████████| 136/136 [00:01<00:00, 134.20it/s]


...

Epoch: 23, loss=0.04699, valid_auc=0.58465, train_auc=0.99928


408it [00:08, 46.75it/s]
100%|██████████| 136/136 [00:01<00:00, 134.95it/s]

Epoch: 24, loss=0.04622, valid_auc=0.57086, train_auc=0.99835



241it [00:05, 43.51it/s]


KeyboardInterrupt: 

In [89]:
model, optimizer = load_checkpoint(os.path.join(root, "results", dataset, "interactE", "weight.pth"), device=device)

Embedding(877, 97)
pretrained finded


In [90]:
pred = predict(model, testloader, device)

100%|██████████| 21/21 [00:00<00:00, 61.72it/s]


In [91]:
pred = pred.detach().cpu().numpy()
# pred = np.round(pred, 3)
pred

array([9.99296665e-01, 9.99974608e-01, 9.99904752e-01, 9.81246769e-01,
       9.41093604e-05, 3.11104268e-05, 9.99999166e-01, 2.24112961e-02,
       9.99894381e-01, 9.99646306e-01, 4.32964180e-06, 9.97670829e-01,
       2.91336537e-03, 9.98132527e-01, 4.62299140e-05, 9.99988914e-01,
       1.76502326e-07, 9.99999762e-01, 1.65335496e-03, 9.99194682e-01,
       1.00000000e+00, 9.99995828e-01, 9.99987245e-01, 3.67666507e-04,
       2.47902626e-05, 9.99567568e-01, 9.99998808e-01, 2.89711033e-06,
       9.99994516e-01, 9.99999881e-01, 9.98152077e-01, 9.99993324e-01,
       4.22160685e-01, 9.99997497e-01, 9.99997377e-01, 9.99977469e-01,
       1.86935576e-05, 9.99999762e-01, 9.99999881e-01, 9.99915004e-01,
       9.99989629e-01, 9.99951839e-01, 9.97748911e-01, 8.24332631e-07,
       7.14886427e-01, 5.42289818e-05, 9.99966025e-01, 3.76991329e-06,
       9.99999642e-01, 2.09501650e-05, 9.99803841e-01, 5.23268938e-01,
       3.84405375e-01, 3.82165535e-06, 9.99978065e-01, 8.77665798e-07,
      

In [92]:
test_df["prob"] = pred
test_df.to_csv("conv_all.csv", index=False)

In [93]:
upload = test_df[["id", "prob"]]
upload.to_csv("upload.csv", index=False)